In [13]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask
import openpyxl
from SPARQLExecutor import SPARQLExecutor

basic_dir = "example"

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [14]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    
    """Базовая функция для связи с сервером"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query(bookquery.subclass_with_parents, entity_id)
        for result in results:
            entities = {}
            entities['subclass'] = result['show']['value'].split('/')[-1]
            entities['subclasslabel'] = result['showLabel']['value']
            entities['classid'] = result['parent']['value'].split('/')[-1]
            entities['classlabel'] = result['parentLabel']['value']
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['classid', 'classlabel' ,'subclass', 'subclasslabel'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
    
def new_sheets(sheetname , ID , filename):
    
    """Функция формирования листов excel"""
    
#     print(sheetname , filename)
    if len(sheetname)>30:
        sheetname = sheetname[:30]
    df = entities_sparqul(ID)
    df = df[df['classid'].isin(list(df.subclass))] # Очищаем от классов без родителей
    df =  df[lambda x : x['subclasslabel'].str[0]!='Q']# Убираем все Q
    local_lst = []
#     for k,v in df.iterrows():# В цикле чистим все повторяющиеся классы и их подклассы
#         if (((v.classid in all_id) or (v.classid in local_lst)) and (v.classid != ID)) or (v.subclass in all_id):
#             local_lst.append(v.subclass)
#             df.drop(k,inplace = True)
    if len(df) == 0:
        return
    new_df = df.groupby(['subclass' , 'subclasslabel']).apply(
        lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()# Формируем столбец классов
    new_df.columns = ['subclass' , 'subclasslabel' , 'class']
    if sheetname == filename or f"{filename}" not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files(f"{basic_dir}/basic"))):
        new_df.to_excel(f'{basic_dir}/basic/{filename}.xlsx', sheet_name = sheetname ,encoding='utf-8-sig' , index = False)
    else:
        with pd.ExcelWriter(f'{basic_dir}/basic/{filename}.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            new_df.to_excel(writer, sheet_name = sheetname , encoding='utf-8-sig' , index = False)

def new_file(name , dic):
    
    """Функция формирования файлов"""
    
    if f'{name}' not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files(f"{basic_dir}/basic"))):
        for k,v in dic.items():
            if k == name:
                new_sheets(k,v , name)
                del(dic[k])
                break
        for k,v in dic.items():
            new_sheets(k,v , name)   
            
def normalization(name , lst):
    executor = SPARQLExecutor()
    labels = executor.execute(bookquery.labels , qid=' '.join(f'wd:{qid}' for qid in lst))
    new_dic = {item['qidLabel']:item['qid'].split('/')[-1] for item in labels}
    return (name , new_dic)

In [15]:
def entities_sparqul_count(entity_id, f = 0):
    
    """Функция обработчик для файла с количеством представителей"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        entities = {}
        raw_results= execute_query(bookquery.subclass_entity_count, entity_id)
        entities['subclass'] = entity_id
        entities['count_P31'] = int(raw_results[0]['entitycount']['value'])
        res.append(entities)
        return pd.DataFrame(data = res, columns = ['subclass', 'count_P31'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul_count(entity_id, f + 1)

In [16]:
def creating_file(file):
    
    """Файлы с представителями"""
    
    filename = file.split('\\')[1].split('.')[0]
    if f"{filename}_with_count" in list(map(lambda x: x.split('\\')[1].split('.')[0] , counter_for_files(f"{basic_dir}/with_count"))):
        return
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        res = []
        df = xl.parse(xl.sheet_names[i])
        for ID in df['subclass']:
            res.append(dask.delayed(entities_sparqul_count)(ID))
        result = dask.compute(*res)
        final_result = pd.concat(result)
        final_result = df.join(final_result.set_index('subclass'), on='subclass').sort_values(
            by = 'count_P31' ,ascending = False)
        if i==0:
            final_result.to_excel(f'{basic_dir}/with_count/{filename}_with_count.xlsx', sheet_name = xl.sheet_names[i] ,encoding='utf-8-sig' , index = False)
        else:
            with pd.ExcelWriter(f'{basic_dir}/with_count/{filename}_with_count.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                final_result.to_excel(writer, sheet_name = xl.sheet_names[i] , encoding='utf-8-sig' , index = False)
       

In [17]:
def without_zeros(file):
    
    """Удаление нулей"""
    
    filename = file.split('\\')[1].split('.')[0]
    if f"{filename}_without_zeros" in list(map(lambda x: x.split('\\')[1].split('.')[0] , counter_for_files(f"{basic_dir}/without_zeros"))):
        return
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        res = []
        df = xl.parse(xl.sheet_names[i])
        df = df[df["count_P31"]>0]
        if len(df)==0:
            continue
        df['signifance'] = ""#Добавляем поле для проверки
        df = df[['subclass' , 'subclasslabel' , 'class' , 'signifance']]
        if i==0:
            df.to_excel(f'{basic_dir}/without_zeros/{filename}_without_zeros.xlsx', sheet_name = xl.sheet_names[i] ,encoding='utf-8-sig' , index = False)
        else:
            with pd.ExcelWriter(f'{basic_dir}/without_zeros/{filename}_without_zeros.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                df.to_excel(writer, sheet_name = xl.sheet_names[i] , encoding='utf-8-sig' , index = False)
        

In [20]:
arr = []
arr.append(normalization("event" , ["Q198" , "Q10931"]))

In [21]:
arr

[('event', {'война': 'Q198', 'революция': 'Q10931'})]

In [ ]:
%%time
#Формирование первичных файлов
res = []
for item in tqdm(arr): 
    print(item[1] , item[0])
    res.append(dask.delayed(new_file)(item[0] , item[1]))
result = dask.compute(*res)

In [ ]:
%%time
#Формирование файла с подсчетом представителей
for file in tqdm(counter_for_files(f"{basic_dir}/basic")):
    creating_file(file)

In [ ]:
%%time
#Финальный этап формирования
res = []
for file in tqdm(counter_for_files(f"{basic_dir}/with_count")):
    res.append(dask.delayed(without_zeros)(file))
result = dask.compute(*res)